<h1>Perform Federated Learning at Grid Platform </h1>
In this notebook, we will train a model using federated approach.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

**NOTE:**
Components:
 - Grid Gateway(http://localhost:8080)
 - Grid Node Bob (http://localhost:3000)
 - Grid Node Alice (http://localhost:3001)
 - Grid Node Bill (http://localhost:3002)

This notebook was made based on <a href="https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2010%20-%20Federated%20Learning%20with%20Secure%20Aggregation.ipynb">Part 10: Federated Learning with Encrypted Gradient Aggregation</a> tutorial

<h2>Import dependencies</h2>

In [4]:
import grid as gr
import syft as sy
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

<h2>Define Model</h2>

In [8]:
hook = sy.TorchHook(th)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc3 = nn.Linear(24, 1)

    def forward(self, x):
        x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)

<h2> Connect to Grid Network</h2>

In [11]:
my_grid = gr.GridNetwork("http://localhost:8080")

<h2>Search a dataset</h2>

In [13]:
data = my_grid.search("#X", "#boston", "#housing")
target = my_grid.search("#Y", "#boston", "#housing")

Match Nodes:  [['Bob', 'http://localhost:3000/'], ['Alice', 'http://localhost:3001/'], ['Bill', 'http://localhost:3002/']]
Match Nodes:  [['Bob', 'http://localhost:3000/'], ['Alice', 'http://localhost:3001/'], ['Bill', 'http://localhost:3002/']]


In [14]:
data

[[(Wrapper)>[PointerTensor | me:98563153569 -> Bob:9775387043]],
 [(Wrapper)>[PointerTensor | me:4729826195 -> Alice:52110458330]],
 [(Wrapper)>[PointerTensor | me:32778334851 -> Bill:92837193854]]]

In [15]:
target

[[(Wrapper)>[PointerTensor | me:2199272751 -> Bob:84773015288]],
 [(Wrapper)>[PointerTensor | me:7833009196 -> Alice:92386388957]],
 [(Wrapper)>[PointerTensor | me:44367517796 -> Bill:75786442740]]]

<h2>Perform Train</h2>

In [17]:
def train(epoch):
    dataset_size = sum([ len(data[i][0]) for i in range(len(data))])
    model.train()
    for i in range(len(data)):
        worker = data[i][0].location
        model.send(worker)
        optimizer.zero_grad()
        pred = model(data[i][0])
        loss = F.mse_loss(pred.view(-1), target[i][0])
        loss.backward()
        optimizer.step()
        model.get()
        loss = loss.get()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * data[i][0].shape[0], dataset_size,
                       100. *  (i * data[i][0].shape[0]) / dataset_size, loss.item()))

for epoch in range(15):
    train(epoch)

Train Epoch: 0 [0/402 (0%)]	Loss: 598.684692
Train Epoch: 0 [134/402 (33%)]	Loss: 497.710236
Train Epoch: 0 [268/402 (67%)]	Loss: 574.146790
Train Epoch: 1 [0/402 (0%)]	Loss: 414.934845
Train Epoch: 1 [134/402 (33%)]	Loss: 117.339111
Train Epoch: 1 [268/402 (67%)]	Loss: 228.963226
Train Epoch: 2 [0/402 (0%)]	Loss: 491.657837
Train Epoch: 2 [134/402 (33%)]	Loss: 345.333344
Train Epoch: 2 [268/402 (67%)]	Loss: 328.528992
Train Epoch: 3 [0/402 (0%)]	Loss: 207.352295
Train Epoch: 3 [134/402 (33%)]	Loss: 26.893274
Train Epoch: 3 [268/402 (67%)]	Loss: 37.589333
Train Epoch: 4 [0/402 (0%)]	Loss: 50.076763
Train Epoch: 4 [134/402 (33%)]	Loss: 177.830841
Train Epoch: 4 [268/402 (67%)]	Loss: 494.693909
Train Epoch: 5 [0/402 (0%)]	Loss: 370.097961
Train Epoch: 5 [134/402 (33%)]	Loss: 199.604370
Train Epoch: 5 [268/402 (67%)]	Loss: 181.044205
Train Epoch: 6 [0/402 (0%)]	Loss: 100.149536
Train Epoch: 6 [134/402 (33%)]	Loss: 25.290924
Train Epoch: 6 [268/402 (67%)]	Loss: 41.039879
Train Epoch: 7 [0/

<h2>Disconnect Grid Nodes</h2>

In [19]:
my_grid.disconnect_nodes()